In [ ]:
import math

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install torchfile

In [ ]:
## Below is the code to extract weights from the VGG_FACE.t7 file given.
## Reference taken from https://github.com/prlz77/vgg-face.pytorch/blob/master/models/vgg_face.py



import torch
import torch.nn as nn
import torch.nn.functional as F
import torchfile

class VGG_16(nn.Module):
    """
    Main Class
    """

    def __init__(self):
        """
        Constructor
        """
        super().__init__()
        self.block_size = [2, 2, 3, 3, 3]
        self.conv_1_1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv_1_2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.conv_2_1 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv_2_2 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv_3_1 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv_3_2 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv_3_3 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv_4_1 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.conv_4_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_4_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_1 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.fc6 = nn.Linear(512 * 7 * 7, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 2622)

    def load_weights(self, path="/content/vgg_face_torch/VGG_FACE.t7"):
        """ Function to load luatorch pretrained
        Args:
            path: path for the luatorch pretrained
        """
        model = torchfile.load(path)
        counter = 1
        block = 1
        for i, layer in enumerate(model.modules):
            if layer.weight is not None:
                if block <= 5:
                    self_layer = getattr(self, "conv_%d_%d" % (block, counter))
                    counter += 1
                    if counter > self.block_size[block - 1]:
                        counter = 1
                        block += 1
                    self_layer.weight.data[...] = torch.tensor(layer.weight).view_as(self_layer.weight)[...]
                    self_layer.bias.data[...] = torch.tensor(layer.bias).view_as(self_layer.bias)[...]
                else:
                    self_layer = getattr(self, "fc%d" % (block))
                    block += 1
                    self_layer.weight.data[...] = torch.tensor(layer.weight).view_as(self_layer.weight)[...]
                    self_layer.bias.data[...] = torch.tensor(layer.bias).view_as(self_layer.bias)[...]

    def forward(self, x):
        """ Pytorch forward
        Args:
            x: input image (224x224)
        Returns: class logits
        """
        x = F.relu(self.conv_1_1(x))
        x = F.relu(self.conv_1_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2_1(x))
        x = F.relu(self.conv_2_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_3_1(x))
        x = F.relu(self.conv_3_2(x))
        x = F.relu(self.conv_3_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_4_1(x))
        x = F.relu(self.conv_4_2(x))
        x = F.relu(self.conv_4_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_5_1(x))
        x = F.relu(self.conv_5_2(x))
        x = F.relu(self.conv_5_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, self.training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, self.training)
        return self.fc8(x)


if __name__ == "__main__":
  model = VGG_16().double()

    # # print(model)
  torch.save(model,'VGG_Face_pytorch.pt')
 

In [ ]:
model = torch.load('VGG_Face_pytorch.pt')

In [ ]:
print(model)

In [ ]:
## Loading weights from pytorch model in list
## list weights_pytorch conatins weights
##list bias_pytorch contains biases
weights_pytorch=[]
bias_pytorch=[]
for l in model.children():
  weights_pytorch.append(l.weight.detach().numpy())
  bias_pytorch.append(l.bias.detach().numpy())

In [ ]:
## Classifier Model to distinguish between Male and Female
## VGG16 model has been used here
## Used Batch Normalization and drop out to increase model accuracy

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras.layers.normalization import BatchNormalization


In [ ]:
model2 = Sequential()
model2.add(Conv2D(input_shape=(64,64,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Flatten())
model2.add(Dense(units=4096,activation="relu"))
model2.add(Dropout(0.5))
model2.add(Dense(units=4096,activation="relu"))
model2.add(Dense(units=1, activation="sigmoid"))

In [ ]:
# parameters for the training of the model
# neural netwrok tries to minimize the loss, loss is relationship to accuarcy 

model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model2.summary()

In [ ]:
# Loading the indexes of trainable layers in list l1
l1=[]
for i in model2.layers:
  if not "batch_normalization" in str(i.name):
    if i.trainable_weights:
      l1.append((model2.layers.index(i)))


In [ ]:
## Transferring weights obtained from the given file to model made in keras(VGG16)
for indx,vgg_l in enumerate(l1[:-3]):

  w = weights_pytorch[indx]
  b = bias_pytorch[indx]
  
  if len(w.shape) < 4:
    w_t = w.transpose(1,0)
  else:
    w_t = w.transpose(2,3,1,0)

  model2.layers[vgg_l].set_weights([w_t, b])
  
  model2.layers[vgg_l].trainable=False

In [ ]:
"""
 Path to given datasets: aligned and valid folders
"""
path_aligned='/content/drive/My Drive/combined/aligned'
path_valid='/content/drive/My Drive/combined/valid'

In [ ]:
# Preprocessing the data
# creating generator to yield training data, validation data in batches
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img


In [ ]:
import random
from PIL import Image
from skimage.transform import rescale, resize, downscale_local_mean
# Mapping for male or female to labels 0 or 1
gender_mapping={'F':0,'M':1}

def list_paths(root_dir):
  """
  This function generates list of image files looping
  through each directory. It generates random 
  files of given sample size from each directory.
  """
  train_paths=[]
  valid_paths=[]
  test_paths=[]

  # Listing all directories in the root path
  dirs=os.listdir(root_dir)

  # Looping through all directories to generate list of files
  for each in dirs:

    # Limiting the number of files using start and end
    files_all = os.listdir(root_dir+"/"+each)
    len_files= len(files_all)
    

    # If there are no files present in the directory, then skip   
    if len_files>0:
      random.shuffle(files_all)
      train_files = files_all[0:int(0.5*len_files)-1]
      valid_files = files_all[int(0.5*len_files):int(0.75*len_files-1)]
      test_files = files_all[int(0.75*len_files):len_files-1]  

      for file in train_files:
        train_paths.append(root_dir+"/"+each+"/"+file)

      for file in valid_files:
        valid_paths.append(root_dir+"/"+each+"/"+file)
       
      for file in test_files:
        test_paths.append(root_dir+"/"+each+"/"+file)

  return train_paths, valid_paths, test_paths

def read_img(path):
  """
  Function to read input image from the path.
  """
  img = load_img(path)
  img_array = img_to_array(img)
  return img_array

def preprocessing(img):
  """
  Function to preprocess the image
  """
  # Resize the image to 64x64
  #img_resized = tf.image.resize(img, (64,64), preserve_aspect_ratio=False)
  img_resized= tf.keras.preprocessing.image.smart_resize(img, (64,64))

  # Normalize image
  img_normalized = img_resized/255.0 #tf.image.per_image_standardization(img_resized)
  
  return img_normalized

def image_generator(files, batch_size):
    """
    Generator function to generate batches of images
    using the list of image paths for training, validation
    and testing.
    """
    while True:
          # Select files (paths/indices) for the batch
          batch_paths  = np.random.choice(a=files, 
                                          size = batch_size)
          batch_input  = []
          batch_output = [] 
          
          # Read in each input, perform preprocessing and get labels
          for input_path in batch_paths:
              input = read_img(input_path)
              output = gender_mapping[input_path.split("/")[-2].split("_")[1]]
            
              input = preprocessing(input)
              batch_input += [ input ]
              batch_output += [ output ]

          # Return a tuple of (input, output) to feed the network
          batch_x = np.array( batch_input )
          batch_y = np.array( batch_output )
          
          yield( batch_x, batch_y )

In [ ]:
## training data, validation data and testing data generated by generator in batches

x_train, x_valid, x_test=list_paths(path_aligned)

In [ ]:
print(len(x_train), len(x_valid), len(x_test))

In [ ]:
#Training of Vgg16 geneder classifier
model2.fit_generator(image_generator(x_train,128),steps_per_epoch = math.ceil(len(x_train)/128),epochs = 25)

In [ ]:
#Evaluation on validation dataset

validation=next(image_generator(x_valid,7210))

In [ ]:
# cross validation
model2.evaluate(x=validation[0], y=validation[1])

In [ ]:
# Evaluation on test set
test=next(image_generator(x_valid,7268))
model2.evaluate(x=test[0], y=test[1])

In [ ]:
# Calling `save('my_model')` creates a SavedModel folder of vgg16 classifier model
from keras.models import load_model
model2.save('my_model.h5')


In [ ]:
#saving vgg16 classifier model weights
model2.save_weights('my_model_weights.h5')


In [ ]:
# loading model
from keras.models import load_model
model2 = load_model("my_model.h5")

In [ ]:
# Prediction of model on test data
y_pred = model2.predict(test[0])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Creates a confusion matrix
y_pred = np.round(model2.predict(test[0]))
y_test=test[1]
cm = confusion_matrix(y_test, y_pred) 

cm_df = pd.DataFrame(cm,
                     index = ['female','male'], 
                     columns = ['female','male'])

plt.figure(figsize=(5.5,4))
sns.heatmap(cm_df, annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import recall_score
# calculate recall
recall = recall_score(y_test, y_pred, average='binary')
print('Recall: %.3f' % recall)

In [ ]:
from sklearn.metrics import precision_score
# calculate prediction
precision = precision_score(y_test, y_pred, average='binary')
print('Precision: %.3f' % precision)

In [ ]:
from sklearn.metrics import f1_score
# calculate score
score = f1_score(y_test, y_pred, average='binary')
print('F-Measure: %.3f' % score)

In [ ]:
! pip install scikit-plot


In [ ]:
# Plot ROC curve
plt.rcParams["figure.figsize"] = (7,7)
from sklearn.metrics import roc_curve, roc_auc_score
predictions=model2.predict(test[0])
fpr, tpr, thresholds = roc_curve(y_test,predictions)
auc_score = roc_auc_score(y_test,predictions)

plt.plot(fpr,tpr,label='Vgg16_model')
plt.plot(fpr,fpr,label='Random guessing')
plt.title('ROC score of Vgg16 = %2.2f'%auc_score)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.show()